> # Candidate Scoring with LLAMA  
***  
In this section, we utilize the information extracted from the PDFs and stored in a JSON file to generate a score for each candidate. The scoring is based on specific features and the job offer they are applying for, allowing us to rank the candidates accordingly.  

`pip install groq python-dotenv`




In [0]:
%pip install groq
# Autres Bibliothèques
%pip install python-dotenv

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


**01.Importing the necessary libraries**


In [0]:
import csv
import re
import os
from groq import Groq
import json
from pprint import pprint
from dotenv import load_dotenv
import io



**02. Load the JSON file that has already been created**

In [0]:
file_path = '/dbfs/mnt/result/resume.json'
with open(file_path, 'r') as file:
    data = json.load(file)  
pprint(data[1:4])

[{'address': 'O 123 Anywhere St., Any City',
  'certifications': None,
  'education': [{'duration': '2010 - 2014',
                 'school': 'BORCELLE SCHOOL Really Great High School'},
                {'duration': '2014 - 2016',
                 'school': 'LICERIA OF TECHNOLOGY Really Great University'}],
  'email': 'hello@reallygreatsite.com',
  'experience': [{'company': 'Really Great Company',
                  'duration': '2016 - Present',
                  'position': 'CREATIVE DIRECTOR',
                  'responsibilities': ['Expert at maintaining relations with '
                                       'corporate clients',
                                       'Responsible for key accounts worth',
                                       'Managed multi-platform regional and '
                                       'national campaigns']},
                 {'company': 'Really Great Company',
                  'duration': '2014 - 2016',
                  'position': 'CREATIVE DIRE

**03. Function to generate the dedicated score for each candidate, structure the results into a CSV file, and store it in Azure Data Lake Gen2**  


In [0]:
def extract_score(full_output):
    """
    Extrait le score de compatibilité d'une chaîne de texte.
    """
    match = re.search(r"\b\d+(\.\d+)?\b", full_output)
    if match:
        return float(match.group(0))
    else:
        print("Score non trouvé dans la réponse.")
        return None


In [0]:
def result_with_llama(api_k, offre, candidat):
    """
    Fonction pour obtenir un score de compatibilité entre une offre et un candidat via l'API Groq (Llama).
    
    Paramètres
    ----------
    api_k : str
        La clé API pour l'authentification avec l'API Llama.
    offre : dict
        Les informations de l'offre d'emploi.
    candidat : dict
        Les informations du candidat.
    
    Retourne
    -------
    float
        Le score de compatibilité entre l'offre et le candidat.
    """
    client = Groq(api_key=api_k)

    prompt = f"""
    Tu es un expert en ressources humaines. J'ai une offre d'emploi et un profil de candidat. 
    Tu dois évaluer le score de compatibilité entre le candidat et l'offre en te basant sur les critères suivants :

    - Compétences : Compare les compétences du candidat avec celles requises par l'offre. Donne plus de poids aux compétences essentielles.
    - Expérience : Vérifie si le candidat a l'expérience minimale requise pour le poste.
    - Certifications : Assure-toi que les certifications du candidat correspondent à celles demandées dans l'offre.
    - Localisation : Évalue si la localisation du candidat est compatible avec le lieu de travail indiqué.
    - Langues : Vérifie si le candidat parle les langues requises dans l'offre.

    Utilise un score de 0 à 10, où 0 signifie que le candidat n'est pas du tout compatible et 10 signifie qu'il est parfaitement compatible avec l'offre. 
    Ne donne pas d'explication, seulement un score.

    Offre d'emploi : 
    {json.dumps(offre, ensure_ascii=False, indent=2)}

    Profil du candidat :
    {json.dumps(candidat, ensure_ascii=False, indent=2)}
    """
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "assistant",
                "content": ""
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
  
    full_output = ""
    for chunk in completion:
        output = chunk.choices[0].delta.content or ""
        full_output += output

    print("Réponse complète de Llama:")
    score = extract_score(full_output)

    return score


In [0]:
load_dotenv()
api_key_llama = os.getenv("API_KEY_LLAMA")

In [0]:
offre = {
    "titre": "Ingénieur DevOps",
    "compétences": ["Docker", "Kubernetes", "CI/CD", "AWS", "Travail en équipe", "Résolution de problèmes"],
    "expérience_minimale": 2,
    "certifications": ["AWS Certified Solutions Architect"],
    "localisation": "France",
    "langues": ["Anglais"]
}

output_file_path = '/mnt/result/candidates_scores.csv'

csv_content = io.StringIO()  
csv_writer = csv.DictWriter(csv_content, fieldnames=[
    "Name", "Email", "Skills", "Experience", "Certifications", "Localisation", "Langues", "Score"
])

for con in data:
    candidat = {
        "name": con.get('name', "Unknown"),
        "email": con.get('email', "Unknown"),
        "skills": con.get('skills', []),
        "experience": con.get('experience', "Unknown"),
        "certifications": con.get('certifications', []),
        "localisation": con.get('address', None), 
        "langues": con.get('langues', [])
    }

    score = result_with_llama(api_key_llama, offre, candidat)

    candidate_row = {
        "Name": candidat["name"],
        "Email": candidat["email"],
        "Skills": ', '.join(str(item) for item in candidat["skills"] if isinstance(item, (str,))) if isinstance(candidat["skills"], list) else str(candidat["skills"]),
        "Experience": candidat["experience"],
        "Certifications": ', '.join(str(item) for item in candidat["certifications"] if isinstance(item, (str,))) if isinstance(candidat["certifications"], list) else str(candidat["certifications"]),
        "Localisation": candidat["localisation"],
        "Langues": ', '.join(str(item) for item in candidat["langues"] if isinstance(item, (str,))) if isinstance(candidat["langues"], list) else str(candidat["langues"]),
        "Score": score
    }
    csv_writer.writerow(candidate_row)
    print(f"NOM : {candidat['name']} | EMAIL : {candidat['email']} | Score de compatibilité : {score}")


try:
    csv_content.seek(0) 
    dbutils.fs.put(output_file_path, csv_content.getvalue(), overwrite=True)
    print(f"CSV file saved successfully at {output_file_path}")
except Exception as e:
    print(f"Error saving CSV file: {e}")


Réponse complète de Llama:
NOM : DANIEL GALLEGO | EMAIL : hello@reallygreatsite.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : CONNOR HAMILTON | EMAIL : hello@reallygreatsite.com | Score de compatibilité : 0.0
Réponse complète de Llama:
NOM : Kayden Zhao | EMAIL : kayden@novoresume.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : Olivia Davis | EMAIL : help@enhancv.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : Sarah Wilson | EMAIL : wilsonn_sarl547@gmail.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : Chloe O'Brian | EMAIL : chloe:obrian@example.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : Aiden Kelly | EMAIL : aiden.kelly@gmail.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : Ahmedd Saaah | EMAIL : hello@ahmedd saaahh.com | Score de compatibilité : 2.0
Réponse complète de Llama:
NOM : RICHARD SANCHEZ | EMAIL : hello@reallygreatsite.com | Score de compatibilité : 